# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import requests, json

# Import API key
from config import g_key

# configure key
gmaps.configure(api_key=g_key)

# cities
from lib import get_city_weather_df

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [14]:
cities_df=get_city_weather_df(1000, force_new=False)
cities_df.head()

Data loaded from cache


,Lng,Lat,City,Country,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,42.547521,-69.659974,port alfred,za,248.38,98,100,7.23,1622314428
1,-3.535401,0.690837,axim,gh,300.56,76,100,7.33,1622314429
2,-148.309941,5.374525,hilo,us,300.33,82,8,5.93,1622314429
3,50.876688,71.751869,belushya guba,ru,272.64,98,100,6.63,1622314430
4,89.088050,-48.883443,busselton,au,281.66,95,100,13.42,1622314430


In [15]:
cities_df.rename(columns={"Lng": "longitude", "Lat": "latitude"}, inplace=True)
cities_df.head()

,longitude,latitude,City,Country,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,42.547521,-69.659974,port alfred,za,248.38,98,100,7.23,1622314428
1,-3.535401,0.690837,axim,gh,300.56,76,100,7.33,1622314429
2,-148.309941,5.374525,hilo,us,300.33,82,8,5.93,1622314429
3,50.876688,71.751869,belushya guba,ru,272.64,98,100,6.63,1622314430
4,89.088050,-48.883443,busselton,au,281.66,95,100,13.42,1622314430


In [16]:
cities_df.describe()

,longitude,latitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1.000000e+03
mean,-1.832621,-0.658573,280.009930,76.581000,66.786000,6.352630,1.622315e+09
std,102.831126,52.826910,21.500725,19.214282,39.626765,3.529759,1.203921e+02
min,-179.732615,-89.998294,213.870000,6.000000,0.000000,0.390000,1.622314e+09
25%,-91.357243,-47.738110,272.347500,71.000000,23.000000,3.637500,1.622315e+09
50%,-1.278046,0.598697,283.245000,80.000000,94.500000,5.985000,1.622315e+09
75%,86.029047,44.743514,297.482500,90.000000,100.000000,8.332500,1.622315e+09
max,179.926128,89.933004,312.040000,100.000000,100.000000,19.760000,1.622315e+09


Looks like temps are in Kelvin.
Lets convert temperature in Kelvin to Celcius.

In [21]:
cities_df['Max Temp'] =  cities_df['Max Temp'] - 273.15

In [22]:
cities_df.describe()

,longitude,latitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1.000000e+03
mean,-1.832621,-0.658573,6.859930,76.581000,66.786000,6.352630,1.622315e+09
std,102.831126,52.826910,21.500725,19.214282,39.626765,3.529759,1.203921e+02
min,-179.732615,-89.998294,-59.280000,6.000000,0.000000,0.390000,1.622314e+09
25%,-91.357243,-47.738110,-0.802500,71.000000,23.000000,3.637500,1.622315e+09
50%,-1.278046,0.598697,10.095000,80.000000,94.500000,5.985000,1.622315e+09
75%,86.029047,44.743514,24.332500,90.000000,100.000000,8.332500,1.622315e+09
max,179.926128,89.933004,38.890000,100.000000,100.000000,19.760000,1.622315e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
# locations = cities_df[['latitude', 'longitude']]
cities_df['coords'] = list(zip(cities_df.latitude, cities_df.longitude))

# locations = gmaps.datasets.load_dataset_as_df('acled_africa')

cities_df.head()

,longitude,latitude,City,Country,Max Temp,Humidity,Cloudiness,Wind Speed,Date,coords
0,42.547521,-69.659974,port alfred,za,248.38,98,100,7.23,1622314428,"(-69.6599737427924, 42.54752062711452)"
1,-3.535401,0.690837,axim,gh,300.56,76,100,7.33,1622314429,"(0.6908368541235603, -3.535400523776815)"
2,-148.309941,5.374525,hilo,us,300.33,82,8,5.93,1622314429,"(5.374525044327214, -148.30994058019303)"
3,50.876688,71.751869,belushya guba,ru,272.64,98,100,6.63,1622314430,"(71.75186881988515, 50.876688188532256)"
4,89.088050,-48.883443,busselton,au,281.66,95,100,13.42,1622314430,"(-48.883442662971376, 89.08804950082055)"


In [18]:
fig = gmaps.figure(map_type='HYBRID')
heatmap_layer = gmaps.heatmap_layer(cities_df.coords)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [39]:
ideal_df = cities_df.copy()
ideal_df = ideal_df[ideal_df['Max Temp'] < 27]
ideal_df = ideal_df[ideal_df['Max Temp'] > 13]
ideal_df = ideal_df[ideal_df['Humidity'] > 20]
ideal_df = ideal_df[ideal_df['Humidity'] < 50]
ideal_df = ideal_df[ideal_df['Cloudiness'] < 70]
ideal_df.dropna(inplace=True)
len(ideal_df)

13

In [40]:
fig = gmaps.figure(map_type='HYBRID')
heatmap_layer = gmaps.heatmap_layer(ideal_df.coords)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [91]:
hotels_data = {
    "Hotel Name": [],
    "City": [],
    "Country": [],
    "Lat": [],
    "Lng": [],
}

In [92]:
def get_place(lat, lon):
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    print('\n\n\n\n', f'circle:5000@{lat},{lon}')
    response = requests.get(
        url +
        f'location={lat},{lon}&radius=5000&type=hotel' +
        '&key='  + g_key
    )
    response.raise_for_status()

    json_data = response.json()
    results = json_data['results']

    if len(results) == 0:
        return None

    result = results[0]    
    return {
        "place_id": result['place_id'],
        "name": result['name'],
        "lat": result['geometry']['location']['lat'],
        "lng": result['geometry']['location']['lng'],
    }


def get_place_detail(place_id):
    url = "https://maps.googleapis.com/maps/api/place/details/json?"
    response = requests.get(
        url +
        f'place_id={place_id}' +
        '&key='  + g_key
    )
    response.raise_for_status()

    json_data = response.json()
    result = json_data['result']
    print('\n\n\n\n', result)
   
    return {
        "place_id": result['place_id'],
        "Hotel Name": result['name'],
        "Lat": result['geometry']['location']['lat'],
        "Lng": result['geometry']['location']['lng'],
        "City": next(filter(lambda x: 'locality' in x['types'] or 'administrative_area_level_1' in x['types'], result['address_components']))['short_name'],
        "Country": next(filter(lambda x: 'country' in x['types'], result['address_components']))['long_name'],
    }

In [93]:
for index, row in ideal_df.iterrows():

    place = get_place(row['latitude'], row['longitude'])
    if place is None:
        continue

    place_detail = get_place_detail(place['place_id'])

    hotels_data["Hotel Name"].append(place_detail['Hotel Name'])
    hotels_data["Lat"].append(place_detail['Lat'])
    hotels_data["Lng"].append(place_detail['Lng'])
    hotels_data["City"].append(place_detail['City'])
    hotels_data["Country"].append(place_detail['Country'])

22 Trans-Canada Hwy, Division No. 5, Subd. E, NL A0K, Canada', 'formatted_phone_number': '(709) 638-3529', 'geometry': {'location': {'lat': 49.3773113, 'lng': -56.9895145}, 'viewport': {'northeast': {'lat': 49.3785276802915, 'lng': -56.9878366697085}, 'southwest': {'lat': 49.3758297197085, 'lng': -56.9905346302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png', 'international_phone_number': '+1 709-638-3529', 'name': 'White Bay Restaurant', 'opening_hours': {'open_now': True, 'periods': [{'close': {'day': 0, 'time': '2200'}, 'open': {'day': 0, 'time': '0700'}}, {'close': {'day': 1, 'time': '2200'}, 'open': {'day': 1, 'time': '0700'}}, {'close': {'day': 2, 'time': '2200'}, 'open': {'day': 2, 'time': '0700'}}, {'close': {'day': 3, 'time': '2200'}, 'open': {'day': 3, 'time': '0700'}}, {'close': {'day': 4, 'time': '2200'}, 'open': {'day': 4, 'time': '0700'}}, {'close': {'day': 5, 'time': '2200'}, 'open': {'day': 5, 'time': '0700'}}, {'close': {'

In [94]:
hotel_df = pd.DataFrame(hotels_data)
hotel_df

,Hotel Name,City,Country,Lat,Lng
0,White Bay Restaurant,NL,CA,49.377311,-56.989514
1,Sarín District,Sarín District,PE,-7.932923,-77.859596
2,Pari,Pari,IR,36.877085,47.417996
3,Techilongo,Techilongo,AO,-15.569041,17.370873
4,NG/47 Concessionaire,North-West District,BW,-19.833535,24.476742
5,Arne Thorbjørn Kristiansen,Innlandet,NO,61.881040,11.215173
6,Shieldsville,Shieldsville,US,44.366074,-93.408833
7,Pencer,Pencer,US,48.699144,-95.637760
8,Gurindji,Gurindji,AU,-17.386853,131.113276


In [49]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'City'

In [ ]:
# Add marker layer ontop of heat map


# Display figure
